In [1]:
import pandas as pd
import os
def modify_float(num):
    try:
        return str(round(float(num),1))
    except:
        return num
def compute_results(df):
    correctness = 0
    sc_correctness = 0
    mad_correctness = 0
    rerailer_correctness = 0
    ltm_correctness = 0
    auto_correctness = 0
    self_check_correctness = 0
    deve_correctness = 0
    for row_index in range(len(df)):
        row = df.iloc[row_index]
        correct_ans = row['Correct_Answer'].lower()
        output_answer = row['Cot_random_answer'].lower()
        sc_answer = row['Smv_Answer'].lower()
        # judge_answer = row['Output_Answer'].lower()
        mad_answer = modify_float(row['MAD_Answer']).lower()
        rerailer_ans = row['Corrected COT Answer'].lower()
        ltm_answer = modify_float(row['ltm_Answer']).lower()
        auto_ans = row['auto_Answer'].lower()
        self_check_ans = modify_float(row['self_check_Answer']).lower()
        deve_ans = modify_float(row['deve_Answer']).lower()
        if correct_ans == output_answer[0]:
            correctness += 1
        else:
            correctness += 0
        if correct_ans == sc_answer[0]:
            sc_correctness += 1
        else:
            sc_correctness += 0
        if correct_ans == mad_answer[0]:
            mad_correctness += 1
        else:
            mad_correctness += 0
        if correct_ans == rerailer_ans[0]:
            rerailer_correctness += 1
        else:
            rerailer_correctness += 0
        if correct_ans == ltm_answer[0]:
            ltm_correctness += 1
        else:
            ltm_correctness += 0
        if correct_ans == auto_ans[0]:
            auto_correctness += 1
        else:
            auto_correctness += 0
        if correct_ans == self_check_ans[0]:
            self_check_correctness += 1
        if correct_ans == deve_ans[0]:
            deve_correctness += 1
    
    result_dict={
        'CoT':round(correctness/len(df),3),
        'SC':round(sc_correctness/len(df),3),
        'FST_CoT':round(mad_correctness/len(df),3),
        'Rerailer':round(rerailer_correctness/len(df),3),
        'LTM':round(self_check_correctness/len(df),3),
        'COVE':round(deve_correctness/len(df),3)
    }
    return result_dict

In [2]:
df = pd.read_csv('../result/sota_comparison/sota_comp.csv')

In [3]:
df.Category.value_counts()

Category
Challenging Math    290
Name: count, dtype: int64

In [4]:
compute_results(df)

{'CoT': 0.421,
 'SC': 0.428,
 'FST_CoT': 0.462,
 'Rerailer': 0.517,
 'LTM': 0.507,
 'COVE': 0.483}

In [6]:
num_of_samples = 1 # 1 question
unit_time = 22*num_of_samples # 22 seconds in average per question when using gpt-4 model
unit_cost = 0.06*num_of_samples # 0.06 USD in average per question when using gpt-4 model
# SC
SC_call = 40

# MAD
num_of_agents = 2
rounds = 3
MAD_call = num_of_agents*rounds

# Rerailer
sc_cot_call = 3 # 3 times of SC
judge_call = 1 # 1 time of Judge
root_checker_call = 10 # Average of 10 steps per question -> 10 times of root checker
debate_call = 3 # 3 times of debate
check_times = 3 # In average, 3 incorrect steps per question
final_cot_call = 1 # 1 time of regernation
consistent_inconsistent_ratio = 322/2477 # Based on the final result, 322 inconsistent out of 2477 questions

derailment_call = sc_cot_call
rerailment_call = (judge_call+root_checker_call+(debate_call)*check_times + final_cot_call)
Rerailer_call = derailment_call + rerailment_call*consistent_inconsistent_ratio
min_call = derailment_call # Derailment only
max_call = (judge_call+root_checker_call+(debate_call+1)*check_times + final_cot_call) + derailment_call # Worst case scenario


# Self-Check
sc_cot = 1
sc_stages = 3
sc_regenerate = 1
self_check_call = root_checker_call*sc_stages + sc_cot + sc_regenerate

# Deductive verification
num_of_ver = 7.504504504504505 # determined from the data distribution provided by author
num_of_check_rounds = 4.738738738738738 # determined from the data distribution provided by author
np = 10
num_of_check_criteria = 3
num_of_check_criteria_per_round = 3
deductive_call = np + np + num_of_ver*num_of_check_rounds*num_of_check_criteria*num_of_check_criteria_per_round



print('CoT time and API calls per question:',1*unit_time,1)
print('SC time and API calls per question:', SC_call*unit_time,SC_call)
print('FSCOT time and API calls per question:',MAD_call*unit_time,MAD_call)
print('LTM time and API calls per question:',self_check_call*unit_time,self_check_call)
print('Rerailer time and API calls per question:',Rerailer_call*unit_time,max_call)
print('CoVE time and API calls per question:',deductive_call*unit_time,deductive_call)




CoT time and API calls per question: 22 1
SC time and API calls per question: 880 40
FSCOT time and API calls per question: 132 6
LTM time and API calls per question: 704 32
Rerailer time and API calls per question: 126.0581348405329 27
CoVE time and API calls per question: 7481.253469685902 340.0569758948137
